<a href="https://colab.research.google.com/github/bythyag/nanoGPT/blob/main/gpt_testbook_bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# in the example karpathy used shakespeare dataset so we will be looking at that only here

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-26 08:03:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-03-26 08:03:46 (26.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
#first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
# since we are training out gpt on next character predicition, we find all the unique characters that occur in this text

#in set no duplicates are allowed, so this means we pick up all the unique characters throught out set() function
print(f"set of unique characters: {set(text)}")

# then we convert the set to a list
print(f"list of unique characters: {list(set(text))}")

# then we sort the list of chars and find the total unique characters in the chars
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"sorted list of unique characters: {chars}")
print(f"unique characters in the text {''.join(chars)}")
print(f"vocab size: {vocab_size}")

set of unique characters: {'t', '\n', '&', 'H', 'P', 'c', ':', '$', '!', 'z', 'B', 'b', 'e', 'm', 'F', 'J', 'I', 'O', 'M', 'j', 'T', 'U', 'g', 'R', 'Z', '3', ',', '-', 'p', "'", 'a', '?', 's', 'o', 'V', 'r', 'y', 'k', 'i', 'A', 'S', 'C', 'x', 'd', 'n', 'E', 'L', 'q', 'G', 'K', '.', 'Q', 'u', 'f', 'h', 'v', ';', 'W', ' ', 'D', 'Y', 'l', 'w', 'X', 'N'}
list of unique characters: ['t', '\n', '&', 'H', 'P', 'c', ':', '$', '!', 'z', 'B', 'b', 'e', 'm', 'F', 'J', 'I', 'O', 'M', 'j', 'T', 'U', 'g', 'R', 'Z', '3', ',', '-', 'p', "'", 'a', '?', 's', 'o', 'V', 'r', 'y', 'k', 'i', 'A', 'S', 'C', 'x', 'd', 'n', 'E', 'L', 'q', 'G', 'K', '.', 'Q', 'u', 'f', 'h', 'v', ';', 'W', ' ', 'D', 'Y', 'l', 'w', 'X', 'N']
sorted list of unique characters: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm

In [ ]:
# create a mapping from characters to integers in a dictionary
# eg: in stoi ch is mapped to i with corresponding character and its index value as we had derived from above
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [ ]:
print(stoi)
print(itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [ ]:
# create encoder-decoder functions

# encoder: take a string, output a list of integers
encode = lambda s: [stoi[c] for c in s]

# decoder: take a list of integers, output a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello, How are you?"))
print(decode(encode("Hello, How are you?")))

[20, 43, 50, 50, 53, 6, 1, 20, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59, 12]
Hello, How are you?


In [ ]:
# we will now encode the entire text dataset and save as pytorch tensor (torch.Tensor)
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
# lets look at the shape and data type of the tensor we have created
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [ ]:
# let's checkout the first 100 tensor values
# the 100 characters we looked at earier will to the GPT look like this
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
# spilt the dataset into testing set and training set
# first 80% will be train, rest val
n = int(0.8*len(data))

#test set
train_data = data[:n]

#training set
val_data = data[n:]

In [ ]:
# length of data
print(f"length of the training data: {len(train_data)}")
print(f"length of the test data: {len(val_data)}")

length of the training data: 892315
length of the test data: 223079


In [ ]:
# for reproduce we use torch manual_seed
torch.manual_seed(1111)

In [ ]:
batch_size = 4 # total independent sequence we can process in parallel
block_size = 8 # maximum context length

# generate a small batch of data of inputs x and targets y
# i.e given first x characters and the corresponding target x+1 character

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
#creating our splits
x_train, y_train = get_batch('train')

print(f"input: {x_train}")
print(f"target: {y_train}")

input: tensor([[47, 53, 52,  8,  0, 27,  1, 58],
        [ 1, 61, 46, 47, 58, 46, 43, 56],
        [ 1, 39, 54, 54, 43, 39, 56,  1],
        [41, 53, 51, 54, 39, 52, 47, 43]])
target: tensor([[53, 52,  8,  0, 27,  1, 58, 47],
        [61, 46, 47, 58, 46, 43, 56,  1],
        [39, 54, 54, 43, 39, 56,  1, 53],
        [53, 51, 54, 39, 52, 47, 43, 42]])


In [ ]:
# for visualisation:

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = x_train[b, :t+1]
        target = y_train[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
    print("---")

when input is [47] the target: 53
when input is [47, 53] the target: 52
when input is [47, 53, 52] the target: 8
when input is [47, 53, 52, 8] the target: 0
when input is [47, 53, 52, 8, 0] the target: 27
when input is [47, 53, 52, 8, 0, 27] the target: 1
when input is [47, 53, 52, 8, 0, 27, 1] the target: 58
when input is [47, 53, 52, 8, 0, 27, 1, 58] the target: 47
---
when input is [1] the target: 61
when input is [1, 61] the target: 46
when input is [1, 61, 46] the target: 47
when input is [1, 61, 46, 47] the target: 58
when input is [1, 61, 46, 47, 58] the target: 46
when input is [1, 61, 46, 47, 58, 46] the target: 43
when input is [1, 61, 46, 47, 58, 46, 43] the target: 56
when input is [1, 61, 46, 47, 58, 46, 43, 56] the target: 1
---
when input is [1] the target: 39
when input is [1, 39] the target: 54
when input is [1, 39, 54] the target: 54
when input is [1, 39, 54, 54] the target: 43
when input is [1, 39, 54, 54, 43] the target: 39
when input is [1, 39, 54, 54, 43, 39] the 

In [ ]:
#lets write our bigram model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1111)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    """
    A simple Bigram-based Language Model that predicts the next token
    based on the current token using an embedding lookup table.
    """
    def __init__(self, vocab_size):
        super().__init__()

        # Token embedding layer (lookup table for next-token logits)
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)

        # Linear layer to allow transformation of logits
        self.output_layer = nn.Linear(vocab_size, vocab_size)

    def forward(self, input_indices, target_indices=None):
        """
        Forward pass for the language model.

        Args:
        - input_indices (Tensor): Tensor of shape (B, T), where B is batch size, and T is sequence length (block size = 8 in our case).
        - target_indices (Tensor, optional): Ground truth token indices for computing loss (same shape as input_indices).

        Returns:
        - logits (Tensor): Predicted logits for the next token (B, T, vocab_size)
        - loss (Tensor, optional): Cross-entropy loss if target_indices is provided.
        """
        # Get token embeddings for the input indices
        logits = self.token_embedding(input_indices)  # Shape: (B, T, vocab_size)
        logits = self.output_layer(logits)  # Shape remains (B, T, vocab_size)

        # Compute loss only if target tokens are provided
        loss = None
        if target_indices is not None:
            B, T, V = logits.shape  # Batch size, Sequence length, Vocabulary size
            logits = logits.view(B * T, V)  # Flatten logits for loss computation
            target_indices = target_indices.view(B * T)  # Flatten targets
            loss = F.cross_entropy(logits, target_indices)  # Compute loss

        return logits, loss

    def generate(self, input_indices, max_new_tokens):
        """
        Generates new tokens based on the input sequence.

        Args:
        - input_indices (Tensor): Initial sequence of token indices (B, T).
        - max_new_tokens (int): Number of tokens to generate.

        Returns:
        - Tensor: Generated sequence including the input indices.
        """
        for _ in range(max_new_tokens):
            # Get model predictions
            logits, _ = self(input_indices)  # (B, T, vocab_size)

            # Focus only on the last timestep's logits
            last_logits = logits[:, -1, :]  # (B, vocab_size)

            # Convert logits to probabilities
            probabilities = F.softmax(last_logits, dim=-1)  # (B, vocab_size)

            # Sample a token from the probability distribution
            next_token = torch.multinomial(probabilities, num_samples=1)  # (B, 1)

            # Append the sampled token to the sequence
            input_indices = torch.cat((input_indices, next_token), dim=1)  # (B, T+1)

        return input_indices


In [ ]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(x_train, y_train)

print(f"logits shape: {logits.shape}")
print(f"loss: {loss}")

print(decode(model.generate(input_indices = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

logits shape: torch.Size([32, 65])
loss: 4.379518508911133

T,oRWeF'tEJvqxL uGqkPWqNMyHRdcmc,f.pdMJcnMts
,L3vOrf&Wwtw:RF.MyBA'!fnWgKeFsg,wq;yHjOlDh
Yji;CYqfKCR'


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100000000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    # check loss over interval
    if steps % 10000 == 0:
        print(f"step: {steps} loss: {loss.item()}")

print(loss.item())

step: 0 loss: 2.4813029766082764
step: 10000 loss: 2.4824483394622803
step: 20000 loss: 2.5528903007507324
step: 30000 loss: 2.4531195163726807
step: 40000 loss: 2.4355146884918213
step: 50000 loss: 2.5187554359436035
step: 60000 loss: 2.4074172973632812
step: 70000 loss: 2.4454853534698486
step: 80000 loss: 2.5549559593200684
step: 90000 loss: 2.5019357204437256
step: 100000 loss: 2.5201737880706787
step: 110000 loss: 2.4098129272460938
step: 120000 loss: 2.4527106285095215
step: 130000 loss: 2.414992570877075
step: 140000 loss: 2.3709492683410645
step: 150000 loss: 2.4506192207336426
step: 160000 loss: 2.4866151809692383
step: 170000 loss: 2.4811582565307617
step: 180000 loss: 2.4354517459869385
step: 190000 loss: 2.288470506668091
step: 200000 loss: 2.4022574424743652
step: 210000 loss: 2.328784465789795


KeyboardInterrupt: 

In [ ]:
print(decode(model.generate(input_indices = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


YOUR:

KERGope cavebral intispr omergdet d,

W:
HEn thaske e
Fistoular ceerth aked, govis wimpay see


## some thoughts
1. loss is stuck at ~ 2.4 after several training and output doesn't make much sense
2. i asked claude for suggestion and it added attention layer which i don't want on this bigram model
3. further asking to improve on the architecture, it suggested to add positional embedding in the layer, layer normalisation, l2 regularisation, and drop out during the training process